# Where's that Buried Chest?

I found not one but **two** buried treasure maps, both pointing to the same spot. Chunkbase confirms there should be swag here (and that's true across all versions that this world has existed). But I literally dug down to bedrock and found bupkiss. **Where's my booty?**

## Imports, Setup and Macros

In [1]:
import json
from collections import Counter
from os import environ
from pathlib import Path
from typing import Any, Collection, Dict, List, Tuple

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
def format_file_size(path: Path) -> str:
    """Print the size of the specified file in
    human-readible form (KB / MB / GB)

    Parameters
    ----------
    path : Path
        The path to the file

    Returns
    -------
    str
        A prettily formatted file size

    Notes
    -----
    I would be shocked if there isn't a utility already built
    into the standard library to do this, but all I could find
    via Googling was a bunch of recipes and examples
    """
    size = path.stat().st_size  # in bytes
    for unit in ("B", "KB", "MB", "GB"):
        if size < 1024 / 2:
            return f"{size:.1f} {unit}"
        size = size / 1024
    return f"{size} TB"

In [3]:
def summarize_keystore(keystore: Dict[str, Any]) -> None:
    """Display a summary of the contents of a key-value store

    Parameters
    ----------
    keystore : dict
        The keystore to summarize

    Returns
    -------
    None
    """

    def _summarize_keystore(keystore: Dict[str, Any]) -> str:
        summary = ""
        for k, v in keystore.items():
            summary += f"\n - `{k}` : "
            if isinstance(v, (str, nbt.TAG_String)):
                summary += f'`"{v}"`'
            elif not isinstance(v, Collection):
                summary += f"`{str(v)}`"
            else:
                length = len(v)
                if 0 < length < 3:
                    summary += "\n"
                    if not isinstance(v, Dict):
                        v = {i: item for i, item in enumerate(v)}
                    summary += "\n".join(
                        (f"\t{line}" for line in _summarize_keystore(v).split("\n"))
                    )
                else:
                    summary += f"({len(v)} items)"
        return summary

    display(Markdown(_summarize_keystore(keystore)))

In [4]:
save_folder = Path(environ["SAVE_PATH"])

# make sure this is set correctly
for path in sorted(save_folder.glob("*")):
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")

- DIM-1 (folder)
- DIM1 (folder)
- advancements (folder)
- data (folder)
- datapacks (folder)
- entities (folder)
- icon.png (8.6 KB)
- level.dat (15.9 KB)
- level.dat_old (15.9 KB)
- level11326231134829879582.dat (0.0 B)
- level14927678268100607923.dat (0.0 B)
- level1786655981796876926.dat (0.0 B)
- level4463453738642305340.dat (0.0 B)
- level6425531070021529407.dat (0.0 B)
- level7443636089696258371.dat (0.0 B)
- level8832581565660323154.dat (0.0 B)
- playerdata (folder)
- poi (folder)
- region (folder)
- serverconfig (folder)
- session.lock (3.0 B)
- stats (folder)


### My coordinates

In [5]:
x, z = (-1783, -4391)

In [6]:
chunk_number = (x // 16, z // 16)
chunk_number

(-112, -275)

In [7]:
region_index = (chunk_number[0] // 32, chunk_number[1] // 32)
region_index

(-4, -9)

In [8]:
chunk_index = (chunk_number[0] % 32, chunk_number[1] % 32)
chunk_index

(16, 13)

## Load up the Chunk Data

In [9]:
region_file = region.RegionFile(
    save_folder / "region" / "r.{}.{}.mca".format(*region_index)
)

In [10]:
chunk_data = region_file.get_chunk(*chunk_index)
summarize_keystore(chunk_data)


 - `xPos` : `-112`
 - `yPos` : `-4`
 - `block_ticks` : (0 items)
 - `isLightOn` : `1`
 - `InhabitedTime` : `50631`
 - `sections` : (24 items)
 - `zPos` : `-275`
 - `blending_data` : 
	
	 - `0` : `"max_section"`
	 - `1` : `"min_section"`
 - `PostProcessing` : (24 items)
 - `Status` : `"full"`
 - `DataVersion` : `3105`
 - `structures` : 
	
	 - `0` : `"References"`
	 - `1` : `"starts"`
 - `fluid_ticks` : (0 items)
 - `Heightmaps` : (4 items)
 - `block_entities` : (0 items)
 - `LastUpdate` : `41297046`

In [11]:
summarize_keystore(chunk_data["structures"]["References"])


 - `minecraft:buried_treasure` : 
	
	 - `0` : `-1176821039216`
 - `minecraft:mineshaft` : 
	
	 - `0` : `-1176821039216`

**It's there!!!!**

(and yes, I already found that mineshalf, lol)

In [12]:
summarize_keystore(chunk_data["structures"]["starts"])


 - `minecraft:buried_treasure` : (5 items)
 - `minecraft:mineshaft` : (5 items)

In [13]:
summarize_keystore(chunk_data["structures"]["starts"][0])


 - `Children` : 
	
	 - `0` : (4 items)
 - `references` : `0`
 - `ChunkZ` : `-275`
 - `id` : `"minecraft:buried_treasure"`
 - `ChunkX` : `-112`

Okay, well I'm in the right chunk. I already knew that.

In [14]:
summarize_keystore(chunk_data["structures"]["starts"][0]["Children"][0])


 - `BB` : (6 items)
 - `id` : `"minecraft:btp"`
 - `O` : `-1`
 - `GD` : `0`

No idea what that "btp" is.

In [15]:
print(chunk_data["structures"]["starts"][0]["Children"][0]["BB"])

[-1783, 57, -4391, -1783, 57, -4391]


I guess "BB" is some sort of boundaries?

Dude, I looked there. And you can even see:

In [16]:
print(chunk_data["block_entities"])

[]


I guess it's possible I alrealy grabbed it and then... reset the chunk? But that somehow didn't restore the chest?

Idk. And in any case, I guess I'm glad at least I can stop looking.